In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install the required ibraries
!pip install -q torch
!pip install -U -q accelerate transformers

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np

In [ ]:
# loading the trained model
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/ASPECT ANALYSIS ALL/BERT_MODEL_FILES/Tokenizer91acc')
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/ASPECT ANALYSIS ALL/BERT_MODEL_FILES/CustomModel91acc")

In [ ]:
# validation file path
VALIDATION_FILE_PATH = '/content/drive/MyDrive/ASPECT ANALYSIS ALL/post split 80 20 ratio train test data/testing_data_20.csv'

In [ ]:
# reading the excel file into a pandas dataframe
validation_data = pd.read_csv(VALIDATION_FILE_PATH)

In [ ]:
# removing the duplicate reviews if any
validation_data = validation_data.drop_duplicates(subset=['Review'])
# deleting the rows with null values if any
validation_data = validation_data.dropna()

In [ ]:
# list of original labels
original_labels = [
    'Adaptability', 'Durability', 'Ease of Use', 'Ergonomics',
    'Interference', 'Performance', 'Use Efficiency', 'Aesthetics',
    'Ease of Reprocessing', 'Ease of Storage', 'Price', 'Safety'
]

In [ ]:
# creating the label to number encoding for each label
labeling_dict = {label: idx for idx, label in enumerate(original_labels)}
# creating the reverse label dictionary to map the predicted numbers back to original labels
labeling_dict_reverse = {idx: label for idx, label in enumerate(original_labels)}

In [ ]:
# converting the reviews
X_val = validation_data["Review"].to_list()

In [ ]:
# tokenizing the reviews
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
# predict function, without GPU predictions

def predict(texts, model, tokenizer, max_length=512, batch_size=8):
    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    model.eval()

    # Make predictions
    predictions = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_inputs = {key: value[i:i+batch_size] for key, value in inputs.items()}
            outputs = model(**batch_inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            _, predicted_labels = torch.max(probabilities, dim=1)
            predictions.extend(predicted_labels.cpu().numpy().tolist())

    return predictions


# Use this function if you have GPU available
def predict_gpu(texts, model, tokenizer, max_length=512, batch_size=8):
    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)

    # Move inputs to GPU
    for key in inputs:
        inputs[key] = inputs[key].to('cuda')

    model.eval()

    # Move model to GPU
    model.to('cuda')

    # Make predictions
    predictions = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_inputs = {key: value[i:i+batch_size].to('cuda') for key, value in inputs.items()}
            outputs = model(**batch_inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            _, predicted_labels = torch.max(probabilities, dim=1)
            predictions.extend(predicted_labels.cpu().numpy().tolist())

    return predictions

In [ ]:
# creating predictions
preds = predict_gpu(X_val, model, tokenizer)

In [ ]:
# storing the predictions and actual labels in a new data frame
val_data_w_preds = pd.DataFrame()
val_data_w_preds['Reviews'] = X_val
# val_data_w_preds['Aspect'] = y_val
val_data_w_preds['Predicted_Aspect'] = preds

In [ ]:
# reverse mapping the numerical predictions to original labels
val_data_w_preds['Aspect'] = validation_data.Aspect.values
val_data_w_preds['Predicted_Aspect'] = val_data_w_preds['Predicted_Aspect'].map(labeling_dict_reverse)

In [ ]:
val_data_w_preds.head()

,Reviews,Predicted_Aspect,Aspect
0,Outstanding stain remover! Works wonders.,Performance,Performance
1,Displays capability but with occasional ineffi...,Performance,Performance
2,The reusable towels hold up well after months ...,Durability,Durability
3,Recycled activewear tears easily and doesn't h...,Use Efficiency,Use Efficiency
4,The TV remote's ergonomic layout makes it easy...,Ergonomics,Ergonomics


In [ ]:
# function to calculate the metrics
def compute_metrics(labels, preds):
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, average='weighted')
    recall = recall_score(y_true=labels, y_pred=preds, average='weighted')
    f1score = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return {"accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1score}

In [ ]:
compute_metrics(val_data_w_preds.Aspect.values,val_data_w_preds.Predicted_Aspect.values)

{'accuracy': 0.9178240740740741,
 'precision': 0.9187643552452461,
 'recall': 0.9178240740740741,
 'f1_score': 0.9180020227796567}

In [ ]:
# individual label wise precision recall metrics
print(classification_report(val_data_w_preds['Aspect'], val_data_w_preds['Predicted_Aspect']))

                      precision    recall  f1-score   support

        Adaptability       0.94      0.90      0.92       141
          Aesthetics       0.92      0.94      0.93       130
          Durability       0.89      0.92      0.91       182
Ease of Reprocessing       0.93      0.95      0.94       121
     Ease of Storage       0.97      0.95      0.96       125
         Ease of Use       0.93      0.89      0.91       133
          Ergonomics       0.90      0.88      0.89       136
        Interference       0.88      0.92      0.90       130
         Performance       0.85      0.85      0.85       231
               Price       1.00      0.95      0.97       128
              Safety       0.91      0.96      0.93       125
      Use Efficiency       0.97      0.95      0.96       146

            accuracy                           0.92      1728
           macro avg       0.92      0.92      0.92      1728
        weighted avg       0.92      0.92      0.92      1728

